# Calculate [user x item] array B for WMF

In [2]:
import os
import collections
from scipy.sparse import csr_matrix
from sys import getsizeof
import numpy as np
import pickle

In [1]:
def create_user_item_matrix(filereader, max_lines=1000):
    user2i = collections.defaultdict(lambda: len(user2i)) # user to index dict
    item2i = collections.defaultdict(lambda: len(item2i)) # item to index dict

    rows = [] # row indices (users) for sparse matrix
    cols = [] # col indices (items) for sparse matrix
    counts = [] # values present at indices given by rows, cols

    for i, line in enumerate(f):
        user, item, count = line.split()
        user_index = user2i[user]
        item_index = item2i[item]
        rows.append(int(user_index))
        cols.append(int(item_index))
        counts.append(int(count))
        if i == max_lines: break
    
    user_item_matrix = csr_matrix((np.array(counts), (np.array(rows), np.array(cols))))
    return user_item_matrix, user2i, item2i

with open('data/taste_profile_subset/train_triplets.txt', encoding='utf8') as f:
    user_item_matrix, user2i, item2i = create_user_item_matrix(f, max_lines=None)

NameError: name 'collections' is not defined

In [3]:
wmf_item2i = pickle.load(open('index_dicts.pkl', 'rb'))
tra2so = {}
with open('data/unique_tracks.txt', encoding='utf8') as f:
    for line in f:
        tra, so = line.split('<SEP>')[0], line.split('<SEP>')[1]
        tra2so[tra] = so
        


In [5]:
print(len(tra2so))
pickle.dump(tra2so, open('track_to_song.pkl', 'wb'))

1000000


In [14]:
print(user_item_matrix.shape)
index_dicts = {'user2i': dict(user2i), 'item2i': dict(item2i)}
pickle.dump(index_dicts, open('index_dicts.pkl', 'wb'))

(1019318, 384546)


In [20]:
from wmf import wmf
pkl_file = open('../user_item_matrix.pkl', 'rb')
B = pickle.load(pkl_file)
S = wmf.log_surplus_confidence_matrix(B, alpha=2.0, epsilon=1e-6)
U, V = wmf.factorize(S, num_factors=50, lambda_reg=1e-5, num_iterations=10, init_std=0.01, verbose=True, dtype='float32', recompute_factors=wmf.recompute_factors_bias)

precompute transpose


  took 5.501 seconds
run ALS algorithm


  iteration 0
    recompute user factors U


    time since start: 172.978 seconds
    recompute item factors V


    time since start: 275.081 seconds
  iteration 1
    recompute user factors U


...

    time since start: 1942.108 seconds
  iteration 7
    recompute user factors U


    time since start: 2103.330 seconds
    recompute item factors V


    time since start: 2236.357 seconds
  iteration 8
    recompute user factors U


    time since start: 2412.011 seconds
    recompute item factors V


    time since start: 2535.263 seconds
  iteration 9
    recompute user factors U


In [21]:
pickle.dump(U, open('../user_wmf_50.pkl', 'wb'))
pickle.dump(V, open('../item_wmf_50.pkl', 'wb'))

In [18]:
os.chdir('code')